<a href="https://colab.research.google.com/github/mirabel-2002/CO2_app/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [111]:
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn

#For encoding the coordinate using location
from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

#For the map
import re
import folium
from IPython.display import HTML

# **Data** **acquisition**



*  Data Sources: primary sources & secondary source
*  Methods of Data Collection:Manual Methods & Automated
   Methods




In [112]:
data = pd.read_csv('transformed_data,.csv')
data

,Name,Location,Dealer Name,Rating,Review Count,Price in SA-Rands (ZAR),Coordinates,Year of Car Model,Car_Age,Price_Category,Dealer_Rating_Ratio,Location_Rating_Average,Review_Count_Category
0,2006 Honda S2000 Base (M6),Newton NJ,Sussex Honda,4.9,1,658750.0,"(41.0582103, -74.7530943)",2006,18,Medium,0.98,4.9,Low
1,2024 Mercedes-Benz EQS 450+ Base,San Jose CA,Mercedes-Benz of Stevens Creek,4.5,2,1705697.5,"(37.3361663, -121.890591)",2024,0,Luxury,0.90,4.5,Low
2,2024 Volkswagen Atlas 2.0T SE,Mankato MN,Mankato Motors,4.7,865,705234.5,"(44.1634663, -93.9993505)",2024,0,Medium,0.94,4.7,High
3,2024 Jeep Wrangler 4xe Sport,Savannah GA,Chrysler Dodge Jeep Ram South Savannah,4.6,212,743302.5,"(43.4129206, -79.7028276)",2024,0,Medium,0.92,4.6,Medium
4,2024 Hyundai Santa Cruz 2.5T XRT,Santa Rosa CA,Manly Hyundai,4.7,1,608809.0,"(38.4404925, -122.7141049)",2024,0,Medium,0.94,4.7,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,2024 Jeep Compass Latitude,Peoria AZ,Moore Chrysler Jeep Fiat,4.7,2,402876.0,"(33.5806115, -112.237294)",2024,0,Low,0.94,4.7,Low
109,2024 Kia EV9 Wind,Rochester NY,Matthews Kia of Greece,4.5,778,1027882.5,"(43.157285, -77.615214)",2024,0,High,0.90,4.5,High
110,2024 Ford F-250 XL,Bellingham WA,Bellingham Ford,4.3,1,898287.0,"(48.7544012, -122.478836)",2024,0,Medium,0.86,4.3,Low
111,2024 Hyundai Venue SEL,Miami Gardens FL,Hyundai of North Miami,4.3,2,367117.5,"(25.9420377, -80.2456045)",2024,0,Low,0.86,4.3,Low


# **Data** **preparation**

 1. *Data* *Collection*
 2. *Data* *Cleaning*
 3. Data Transformation
 4. Data Integration
 5. Data Reduction




## **Drop** **column**(s)

In [113]:
deleteCol =['Coordinates','Year of Car Model','Car_Age',	'Price_Category',	'Dealer_Rating_Ratio',
            'Location_Rating_Average',	'Review_Count_Category']

In [114]:
df = data.drop(deleteCol, axis=1)


## **Data** **transformation**

To perform data transformation on the provided dataset, we can take the following steps:



*  Standardize column names: Ensure all column names are
   consistent and follow a standard naming convention.
*  Convert currency: Change the price from South African
   Rands (ZAR) to US Dollars (USD).
*   Normalize review counts: Scale the review counts for
    easier comparison.
*   Extract year and model: Separate the year and model from
    the "Name" column.




In [121]:

def data_transformation(df):

  database = pd.DataFrame(df)

  # 1-Standardize column names:
  database.columns = ['Names', 'Location', 'Dealer_Name', 'Rating', 'Review_Count', 'Price_ZAR']

  # 2-Parsing: Extract string from existing columns names to create new columns:

  database['Year'] = database['Names'].apply(lambda x: x.split()[0])
  database['Year'] = database['Year'].astype(int) # Convert 'Year' to integer

  database['Brand'] = database['Names'].apply(lambda x: x.split()[1])

  #database['Model'] = database['Names'].apply(lambda x: x.split()[2:])
  database["Model"] = database["Names"].apply(lambda x: ' '.join(x.split()[2:]))

  database['State'] = database['Location'].apply(lambda x: x.split()[1])
  database['City'] = database['Location'].apply(lambda x: x.split()[0])

  # 3-Deriving: Convert currency from ZAR to USD (assuming 1 ZAR = 0.067), age of car
  database['Price_USA'] = database['Price_ZAR']*0.067
  cur_year = 2014
  database['Car_Age'] = cur_year - database['Year']

  # 4-Normalization: rating & review counts (eg. by normalization by dividing by max review count)
  max_review = database['Review_Count'].max()
  database['Normalized_Review_Count'] = database['Review_Count']/max_review

  max_rating = database['Rating'].max()
  database['Dealer_Rating_Ratio'] = database['Rating']/max_rating

  # 5-Binnig price, rating:

  price_bins = [0,500000,1000000,1500000, float('inf')]
  price_lables = ['Low','Medium','High','Luxury']
  database['Price_Category'] = pd.cut(database['Price_ZAR'], bins=price_bins, labels=price_lables)

  rating_bins = [0,2,4,5, float('inf')]
  rating_lables = ['Poor', 'Good', 'Excellient','Brilliant']
  database['Rating_Category'] = pd.cut(database['Rating'], bins=rating_bins,labels=rating_lables)


  # 6-Aggregation: This process allows you to see the average rating for each location, providing
  # insights into how dealers in different locations are rated on average.

  avg_rating = database.groupby('Location')['Rating'].mean()
  database['Location_Rating avg'] = database['Location'].map(avg_rating)


  # 7-NB!!!! Replacing columns:
  database.drop(columns = ['Price_USA'], inplace=True)

  # 8-Encoding: Geocoding the location to create a coordination column:

  geolocator = Nominatim(user_agent = 'my_geocoder') # Make sure you have a descriptive user agent
  #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

  # Define a function to handle geocoding and extract coordinates
  def get_coordinates(location):
    try:
      geo_result = geolocator.geocode(location)
      if geo_result:
        return (geo_result.latitude, geo_result.longitude)

      else:
          return None

    except(GeocoderTimedOut, GeocoderUnavailable) as e:
      print(f'Geocoding failed for Location: {location}. Error:{e}')
      return None
  database['Coordinates'] = data['Location'].apply(get_coordinates)

#standardization: scaling data to have a mean of 0 and a standard deviation of 1
#Discretization: converting continuous data into discrete intervals
#pivoting: rotating data from rows to columns or vice versa
#merging: combining data from different sources into a single dataset
#filtering
# Data type conversion
#imputation: filling in missing value with estimated values


  return database




In [ ]:
database = data_transformation(df)
database

Streaming output truncated to the last 5000 lines.
    raise ConnectionError(e, request=request)
requests.exceptions.ConnectionError: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=++Centennial+CO&format=json&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x78a4a1100cd0>: Failed to establish a new connection: [Errno 111] Connection refused'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
  File "/usr/local/lib/python3.10/dist-packages/geopy/extra/rate_limiter.py", line 274, in __call__
    res = self.func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/geopy/geocoders/nominatim.py", line 297, in geocode
    return self._call_geocoder(url, callback, timeout=timeout)
  File "/usr/l